<a href="https://colab.research.google.com/github/naokishibuya/cuda-q-experiments/blob/main/notebooks/04_qaoa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 — QAOA (Quantum Approximate Optimazation Algorithm)

## QAOA and Ising/QUBO

Many combinatorial optimization problems can be expressed in QUBO (Quadratic Unconstrained Binary Optimization) format, which can be mapped to an **Ising Hamiltonian**:

$$
H = \sum_{i<j} J_{ij} Z_i Z_j + \sum_i h_i Z_i
$$

The goal is to find the ground state (minimum energy), which corresponds to the optimal solution.
- QAOA uses this $H$ as the "cost Hamiltonian", and alternates it with a **mixer** $\sum_i X_i$.
- This makes the overall circuit equivalent to simulating the **transverse-field Ising model (TFIM)** for $p$ steps.


In [1]:
# Uncomment and install cuda-q if not already

# %pip install --upgrade pip
# %pip install cudaq

## Cost Hamiltonian and Mixer Hamiltonian

For our two-qubit problem, we implement a minimal **QAOA** for the Hamiltonian

$$
H = Z_0 Z_1 + X_0 + X_1
$$

with $p\in\{1,2\}$.

We split $H$ into commuting pieces used by QAOA:

$$
H_Z = Z_0 Z_1, \quad H_X = X_0 + X_1
$$

We only use $H$ and not $H_Z$ and $H_X$ individually but conceptually, the splitting allow us to exploit commutation and approximate the evolution under H.

In [2]:
import cudaq

HZ = cudaq.spin.z(0) * cudaq.spin.z(1)
HX = cudaq.spin.x(0) + cudaq.spin.x(1)
H  = HZ + HX
print('H =', str(H))


H = (1+0i) * X0 + (1+0i) * X1 + (1+0i) * Z0Z1


## QAOA Ansatz

We approximate each observable using an ansatz.

Setup:
- State prep is $|+\rangle^{\otimes 2}$.
- For each layer $\ell=1\dots p$, we apply:
$$ e^{-i\,\gamma_\ell H_Z}\;e^{-i\,\beta_\ell H_X} $$

Implementations:
- $e^{-i\,\gamma Z_0Z_1} = \mathrm{CNOT}_{0\to1}\,R_z(2\gamma)_1\,\mathrm{CNOT}_{0\to1}$
- $e^{-i\,\beta (X_0+X_1)} = R_x(2\beta)\otimes R_x(2\beta)$


In [3]:
import numpy as np

# Ansatz for Cost Hamiltonian: ZZ rotation
def cost_layer(k, q0, q1, gamma):
    k.cx(q0, q1)
    k.rz(2*gamma, q1)
    k.cx(q0, q1)

# Ansatz for Mixer Hamiltonian: X ⊗ X
def mixer_layer(k, q0, q1, beta):
    k.rx(2*beta, q0)
    k.rx(2*beta, q1)

# QAOA Ansatz
def qaoa_kernel(p):
    # params: gammas (p), betas (p)
    k, *params = cudaq.make_kernel(*([float]*(2*p)))
    q = k.qalloc(2)
    # |++>
    k.h(q)

    # unpack and
    gammas = params[:p]
    betas  = params[p:]
    for l in range(p):
        cost_layer(k, q[0], q[1], gammas[l])
        mixer_layer(k, q[0], q[1], betas[l])
    return k

# Cost function
def energy(x, p):
    k = qaoa_kernel(p)
    return cudaq.observe(k, H, *[float(v) for v in x]).expectation()


## Optimize $(\boldsymbol{\gamma},\boldsymbol{\beta})$ for $p=1,2$

We optimize $(\boldsymbol{\gamma},\boldsymbol{\beta})$ and compare to the exact ground energy $-\sqrt{5}$.


In [4]:
from scipy.optimize import minimize
from math import pi

def solve_p(p):
    # Initialize near reasonable angles
    g = (np.arange(1, p+1)/p) * (np.pi/2)     # for cost
    b = (1 - (np.arange(0, p))/p) * (np.pi/4) # for mixer
    x0 = np.concatenate([g, b])

    result = minimize(
        lambda x: energy(x, p),
        x0=x0,
        method='Nelder-Mead',
        options={
            'maxiter': 1000,
            'xatol':1e-10,
            'fatol':1e-10,
        }
    )

    nit = result.nit # number of iterations

    if not result.success:
        print(result.message)
        return np.nan, np.nan, nit

    optimal_x = result.x;
    optimal_e = energy(optimal_x, p)
    return optimal_x, optimal_e, nit


print(f"Ground energy: -sqrt(5) ≈ {-np.sqrt(5):.8f}")

np.set_printoptions(formatter={'float': '{:0.4f}'.format})
for p in [1, 2, 3]:
    optimal_x, optimal_e, nit = solve_p(p)
    print(f"p={p}: Minimum E ≈ {optimal_e:.8f}, params = {optimal_x}, nit = {nit}")


Ground energy: -sqrt(5) ≈ -2.23606798
p=1: Minimum E ≈ -2.23606813, params = [1.3390 1.1781], nit = 62
p=2: Minimum E ≈ -2.23606837, params = [0.3286 2.1228 0.7448 0.4484], nit = 123
p=3: Minimum E ≈ -2.23606890, params = [0.6458 0.5547 1.7702 0.7690 0.5106 0.2835], nit = 157


The result shows only 1 layer is enough to approximate the ground energy. Adding more layers does not help and add more iterations to converge.